# Olympic games medals analysis


In this notebook we will analyze `olimpic_medals.csv` dataset for pandas skill training.



<sub><sup>Homework for EPAM Training Center, Nizhny Novgorod, Russia, 2020</sup></sub>

### First install useful libs and make imports

In [ ]:
%pip install -r requirements.txt

In [ ]:
import pandas as pd
from iso3166 import countries

from codes_mapping import ioc2iso

### Then load dataset and take quick look at data

In [ ]:
dset = pd.read_csv('olimpic_medals.csv')
dset.info()

In [ ]:
dset.head(3)

### Define some useful functions

In [ ]:
def get_country_name(alpha3code: str) -> str:
    """Map codes between IOC and ISO formats according to Wikipedia article
    (https://en.wikipedia.org/wiki/Comparison_of_alphabetic_country_codes), exclude 
    former countries and former country names. Return regular country name (from ISO-3166).   
    
    Args:
      alpha3code: three-letter country code
      
    Returns:
      regular country name"""
    
    try:
        return countries.get(ioc2iso.get(alpha3code, alpha3code)).name
    
    except KeyError:
        return alpha3code

### Analysis

#### 1. How many medals did Jesse Owens get in 1936 year?

Since athlete names are stored as pair of "family name / first name", let's switch name and family name in condition. Thus we are searching 'OWENS, Jesse' now. 
However usually is better to normalize strings to lowercase when searching, that's what we do.

In [ ]:
mask_year1936 = dset.Edition == 1936
mask_jesse = dset.Athlete.str.lower() == 'Owens, Jesse'.lower()

cnt = len(dset[(mask_year1936 & mask_jesse)])

f'Jesse Owens got {cnt} medal(s) in 1936' 

#### 2. Which country has won more gold medals in Badminton (men) than others?

In [ ]:
mask_medal = dset.Medal == 'Gold'
mask_gender = dset.Event_gender == 'M'
mask_badminton = dset.Discipline == 'Badminton'
 
alpha3code = dset[(mask_medal & mask_gender & mask_badminton)].NOC.value_counts().idxmax()  # country code
country_name = get_country_name(alpha3code)

f'Men of {country_name} ({alpha3code}) have won more gold medals in Badminton than others.'

#### 3. Which three countries have won more medals  than others from 1984 to 2008?

In [ ]:
mask_years = dset.Edition >= 1984
top3 = dset[mask_years].groupby('NOC').count().sort_values(by='Medal', ascending=False)['Medal'][:3]
top3.index = top3.index.map(lambda x: f'{get_country_name(x)} ({x})')

top3

#### 4. Men who won gold medals in 100m sprint

In [ ]:
mask_medal = dset.Medal == 'Gold'
mask_gender = dset.Event_gender == 'M'
mask_100m = dset.Event == '100m'

show_cols =['City', 'Edition', 'Athlete', 'NOC']
sprint_winners = dset[(mask_medal & mask_gender & mask_100m)][show_cols].sort_values(by='Edition',
                                                                                     ascending=False)

sprint_winners.NOC = sprint_winners.NOC.map(lambda x: f'{get_country_name(x)} ({x})')

sprint_winners

#### 5.1 How many medals did men and women win? 

In [ ]:
mask_gender_m = dset.Gender == 'Men'
mask_gender_w = dset.Gender == 'Women'

print(f'Men won {len(dset[mask_gender_m])} medals\nWomen won {len(dset[mask_gender_w])} medals')

#### 5.2 How many gold, silver and bronze medals have won each gender?

In [ ]:
medal_distribution = dset.groupby(['Gender', 'Medal'])['Medal'].agg('count')
medal_distribution

In [ ]:
medal_distribution.plot.pie()

In [ ]:
medal_distribution.plot.bar()

#### 6. Count of medals on each Olympics

In [ ]:
cols = { 
    'Bronze': dset[dset.loc[:, 'Medal'] == 'Bronze'].groupby('Edition').agg('count')['Medal'],
    'Silver': dset[dset.loc[:, 'Medal'] == 'Silver'].groupby('Edition').agg('count')['Medal'],
    'Gold': dset[dset.loc[:, 'Medal'] == 'Gold'].groupby('Edition').agg('count')['Medal'],
}

xticks = sorted(list(set(dset.Edition)))

pd.DataFrame(cols).plot(figsize=(15,5), xticks=xticks, grid=True)

#### 7. Count of medals by countries

In [ ]:
medal_count = dset.groupby('NOC')['Medal'].agg('count')
medal_first = dset.groupby('NOC')['Edition'].agg('min')
medal_last = dset.groupby('NOC')['Edition'].agg('max')

result = pd.DataFrame([medal_count, medal_first, medal_last]).T
result.columns = ['Count', 'First', 'Last']
result.index = result.index.map(lambda x: f'{get_country_name(x)} ({x})')

pd.set_option('display.max_rows', len(result))
result

#### 8. Athletes on 200m or 100m distances with medals in Beijing

In [ ]:
mask_beijing = dset.City == 'Beijing'
mask_discipline = (dset.Event == '100m') | (dset.Event == '200m')
dset[mask_beijing & mask_discipline]

#### 9. USA gold medals for men and women in Athletics

In [ ]:
mask_athletics = dset.Discipline == 'Athletics'
mask_medal = dset.Medal == 'Gold'
mask_usa = dset.NOC == 'USA'
mask_m = dset.Gender == 'Men'
mask_w = dset.Gender == 'Women'

cols = {
    'Men': dset[mask_athletics & mask_medal & mask_usa & mask_m].groupby('Edition').agg('count')['Medal'],
    'Women': dset[mask_athletics & mask_medal & mask_usa & mask_w].groupby('Edition').agg('count')['Medal']
}

pd.DataFrame(cols).plot(kind='bar', figsize=(15,5))

#### 10. Top 5 athletes by gold medal count

In [ ]:
dset[dset.Medal == 'Gold'].groupby('Athlete').agg('count')['Medal'].sort_values(ascending=False)[:5].plot(kind='barh')

#### 11. Count medals on last Olympics

In [ ]:
last_year = dset.Edition.max()
medal_count = dset[dset.Edition == last_year].agg('count')['Medal']

f'Total medals got in {last_year} year is {medal_count}.'

#### 12. Top US athletes by years

In [ ]:
us_only = dset[dset.loc[:, 'NOC'] == 'USA']

year_groups = us_only.groupby(['Edition', 'Athlete', 'Discipline']).agg('count').reset_index()

result = year_groups.loc[year_groups.groupby('Edition')['Medal'].idxmax()]
result = result.loc[:, ['Edition', 'Athlete', 'Discipline', 'Medal']]
result.columns = ['Year', 'Athlete', 'Discipline', 'Medal_count']

result